In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%%bash
# Logistics #2: install the transformers package, create a folder, download the dataset and a patch
pip -q install transformers

# remove the directory if necessary
# rm -rf "/content/gdrive/MyDrive/6864_hw4/"

mkdir "/content/drive/My Drive/NLP/"
cd "/content/drive/My Drive/NLP/"

# wget http://nlp.stanford.edu/data/glove.6B.zip
# unzip glove.6B.zip
# ls
# pwd

mkdir: cannot create directory ‘/content/drive/My Drive/NLP/’: File exists


In [3]:
import numpy as np
import pandas as pd
import torch

if torch.cuda.is_available():
  device = torch.device('cuda')

  print('There are %d GPU(s) available.' % torch.cuda.device_count())

  print('We will use the GPU:', torch.cuda.get_device_name(0))

In [8]:
project_path = '/content/drive/My Drive/NLP/'
dataset = pd.read_csv(project_path + 'fake_and_real_news/combined.csv')
dataset['text'] = dataset['text'].str.replace("^.*\(Reuters\) - ", "", regex=True)

In [9]:
dataset.head()

,text,label
0,President Trump made a joke while speaking to ...,0.0
1,"Tuesday night, retired neurosurgeon and former...",0.0
2,This is a very big development. We all knew th...,0.0
3,President Barack Obama will ask the U.S. Congr...,1.0
4,Beyonce made an attempt to glorify the violent...,0.0


In [10]:
from keras.models import Sequential,load_model
from keras.layers import Dense,Dropout
from keras.layers import LSTM,Conv1D
from keras.layers import MaxPooling1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras import optimizers
from keras.layers import TimeDistributed
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pickle

from sklearn.preprocessing import LabelEncoder

In [11]:
#Clean the text 
import re
import string
from nltk.corpus import stopwords
from nltk import re, SnowballStemmer

def clean_text(text):
    import nltk
    nltk.download('stopwords')
    translate_table = dict((ord(char), None) for char in string.punctuation)
    text = text.translate(translate_table)

    re_url = re.compile(r"((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\
                        .([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*",
                        re.MULTILINE | re.UNICODE)
    re_ip = re.compile("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}")

    text = re_url.sub("URL", text)

    text = re_ip.sub("IPADDRESS", text)

    text = text.lower().split()

    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]

    text = " ".join(text)

    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)


    return text

In [ ]:
exts=[]
texts=dataset['text']#####################################
label=dataset['label']
#print(label)
#X=texts.astype(str).values.tolist()
#X=np.reshape(X,(-1,1))
# from DataPrep.Clean_Texts import clean_text
X=texts.map(lambda x: clean_text(x))
#print(X)
#label=label.astype(int).values


In [13]:
labelEncoder=LabelEncoder()
encoded_label=labelEncoder.fit_transform(label)
y=np.reshape(encoded_label,(-1,1))

In [14]:
training_size=int(0.8*X.shape[0])
X_train=X[:training_size]
y_train=y[:training_size]
X_test=X[training_size:]
y_test=y[training_size:]
#print(X_test)

#print(len(X_train),len(X_test))

#Max no of Vocab
vocabulary_size = 400000

time_step=300
embedding_size=100
#Tokenizing texts
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(X_train)
sequences_train= tokenizer.texts_to_sequences(X_train)
X_train = sequence.pad_sequences(sequences_train, maxlen=time_step,padding='post')

print(len(tokenizer.word_index))


vocab_size=len(tokenizer.word_index)+1

173886


In [15]:
#Reading Glove
f = open(project_path + 'glove.6B.100d.txt',encoding='utf-8')
embeddings={}
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings[word] = coefs
f.close()

print('Total %s word vectors.' % len(embeddings))

Total 400000 word vectors.


In [16]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((vocab_size, embedding_size))
for word, i in tokenizer.word_index.items():
	embedding_vector = embeddings.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

print(embedding_matrix.shape[0],embedding_matrix.shape[1])

173887 100


In [17]:
sequences_test= tokenizer.texts_to_sequences(X_test)
X_test = sequence.pad_sequences(sequences_test, maxlen=time_step,padding='post')

# Embedding
#maxlen = 100
#embedding_size = 32

# Convolution
filter_length = 3
nb_filter = 128

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=time_step,
                    weights=[embedding_matrix],trainable=False))
#model.add(Dropout(0.4))

#model.add(LSTM(128))

model.add(Conv1D(filters=nb_filter,
                        kernel_size=filter_length,
                        activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

#model.add(Dense(units=128,activation='relu'))
#model.add(Dropout(rate=0.8))



model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

In [19]:
model.fit(X_train,y_train,batch_size=64,epochs=10)


print("Saving Model...")
model_name = 'Models/Model_cnn_FR_2.h5'########################################3
model.save(model_name)#################################################

Epoch 1/10
562/562 [==============================] - 55s 97ms/step - loss: 0.2774 - accuracy: 0.8814
Epoch 2/10
562/562 [==============================] - 55s 98ms/step - loss: 0.0442 - accuracy: 0.9863
Epoch 3/10
562/562 [==============================] - 55s 98ms/step - loss: 0.0157 - accuracy: 0.9961
Epoch 4/10
562/562 [==============================] - 56s 100ms/step - loss: 0.0081 - accuracy: 0.9987
Epoch 5/10
562/562 [==============================] - 56s 100ms/step - loss: 0.0071 - accuracy: 0.9983
Epoch 6/10
562/562 [==============================] - 56s 99ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 7/10
562/562 [==============================] - 56s 99ms/step - loss: 7.7769e-04 - accuracy: 1.0000
Epoch 8/10
562/562 [==============================] - 56s 99ms/step - loss: 4.4338e-04 - accuracy: 1.0000
Epoch 9/10
562/562 [==============================] - 56s 99ms/step - loss: 3.1588e-04 - accuracy: 1.0000
Epoch 10/10
562/562 [==============================] - 56s 99ms/step

In [20]:
score=model.evaluate(X_test,y_test,verbose=1)
print('acc: '+str(score[1]))

from sklearn.metrics import precision_recall_fscore_support,classification_report
y_pred=model.predict_classes(X_test)
print('Classification report:\n',classification_report(y_test,y_pred))
#print('Classification report:\n',precision_recall_fscore_support(y_test,y_pred))
#print(y_pred)
'''
model = load_model('Models/Model_cnn_FR_2.h5')
model.name='Model_cnn_FR_2.h5'
'''

from sklearn.metrics import precision_recall_fscore_support, classification_report
y_pred=model.predict_classes(X_test)
print('Classification Report: '+classification_report(y_test, y_pred))


281/281 [==============================] - 6s 21ms/step - loss: 0.0583 - accuracy: 0.9878
acc: 0.9877505302429199


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Classification report:
               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4836
           1       0.99      0.99      0.99      4144

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

Classification Report:               precision    recall  f1-score   support

           0       0.99      0.99      0.99      4836
           1       0.99      0.99      0.99      4144

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980



Below is the same thing except training with 1 epoch


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_size, input_length=time_step,
                    weights=[embedding_matrix],trainable=False))
#model.add(Dropout(0.4))

#model.add(LSTM(128))

model.add(Conv1D(filters=nb_filter,
                        kernel_size=filter_length,
                        activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())

#model.add(Dense(units=128,activation='relu'))
#model.add(Dropout(rate=0.8))



model.add(Dense(1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',
          optimizer='adam',
          metrics=['accuracy'])

In [ ]:
model.fit(X_train,y_train,batch_size=64,epochs=1)


print("Saving Model...")
model_name = 'Models/Model_cnn_FR_2.h5'########################################3
model.save(model_name)#################################################

562/562 [==============================] - 55s 97ms/step - loss: 0.2176 - accuracy: 0.9025
Saving Model...


In [ ]:
score=model.evaluate(X_test,y_test,verbose=1)
print('acc: '+str(score[1]))

from sklearn.metrics import precision_recall_fscore_support,classification_report
y_pred=model.predict_classes(X_test)
print('Classification report:\n',classification_report(y_test,y_pred))
#print('Classification report:\n',precision_recall_fscore_support(y_test,y_pred))
#print(y_pred)
'''
model = load_model('Models/Model_cnn_FR_2.h5')
model.name='Model_cnn_FR_2.h5'
'''

from sklearn.metrics import precision_recall_fscore_support, classification_report
y_pred=model.predict_classes(X_test)
print('Classification Report: '+classification_report(y_test, y_pred))

281/281 [==============================] - 6s 20ms/step - loss: 0.0392 - accuracy: 0.9871
acc: 0.9870824217796326


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


Classification report:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99      4836
           1       0.97      1.00      0.99      4144

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

Classification Report:               precision    recall  f1-score   support

           0       1.00      0.98      0.99      4836
           1       0.97      1.00      0.99      4144

    accuracy                           0.99      8980
   macro avg       0.99      0.99      0.99      8980
weighted avg       0.99      0.99      0.99      8980

